# Point Cloud Demonstrations

## Registration

Registers two point clouds with slight offset.


In [1]:
import open3d as o3d
import numpy as np
import copy
import random

import utils

In [ ]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target])

# load point clouds
base = o3d.io.read_point_cloud("./Point Clouds/Cleaner/1.ply")
source = o3d.io.read_point_cloud("./Point Clouds/Cleaner/8.ply")

# get 4x4 transformation matrix
trans = utils.register_clouds(base, source)

# show original clouds
o3d.visualization.draw_geometries([base, source])

# show aligned clouds
draw_registration_result_original_color(source, base, trans)


## Normals of point cloud
Shows the histogram of normals for the cloud. See utils module - running in notebook is much slower

## Outlier removal
Removes outliers and scatter artifacts from the pointcloud

In [ ]:
cl, ind=utils.clean_cloud("./Point Clouds/Cleaner/1.ply", view=True)

## Cleaned and combined 8 clouds

In [ ]:
base = o3d.io.read_point_cloud("./Point Clouds/combined_cloud_clean.ply")
o3d.visualization.draw_geometries([base])

# Region Growing Segmentation
Demonstrates region growing algorithm for downsampled cloud

In [8]:
cl = o3d.io.read_point_cloud("./Point Clouds/combined_cloud_clean.ply")
pcd = cl.voxel_down_sample(0.01)

planes_list = utils.region_grow(pcd)

for plane in planes_list:
    colour = [random.uniform(0,1),random.uniform(0,1),random.uniform(0,1)]
    for i in plane:
        pcd.colors[i] = colour

o3d.visualization.draw_geometries([pcd])

# extract walls which is the largest object in the cloud

walls = planes_list[np.argmax(np.array([len(i) for i in planes_list]))]
walls = pcd.select_down_sample(walls)

## Isolating each wall plane

In [15]:
planes_list = utils.region_grow(walls, 0.8, True)

colours = [[1,0,0],[0,1,0],[0,0,1]]

for plane, colour in zip(planes_list, colours):
    for i in plane:
        walls.colors[i] = colour

o3d.visualization.draw_geometries([walls])

# get normals for each plane
plane_normals = []
for plane in planes_list:
    # make a copy otherwise points a deleted from selection
    wall_cp = copy.deepcopy(walls)
    plane_points = np.asarray(wall_cp.select_down_sample(plane).points)
    
    pseudoinverse = np.linalg.pinv(plane_points.T)
    plane_normals.append(pseudoinverse.T.dot(np.ones(plane_points.shape[0])))
    
plane_normals = np.array(plane_normals)
print(plane_normals)


[[ 0.95297908  0.27252682 -0.91992971]
 [ 0.156436   -4.40440385 -1.11348369]
 [-0.94290001  0.22964552 -1.01562925]]


## SVD of Walls
Principle vectors don't seem aligned as I thought they would be?

In [23]:
plane = planes_list[2]
wall_cp = copy.deepcopy(walls)
plane = wall_cp.select_down_sample(plane)
points = np.asarray(plane.points)
u, s, v = np.linalg.svd(points)

s /= np.linalg.norm(s)

for i in range(len(s)):
    v[i] *= s[i]

o3d.visualization.draw_geometries([plane, utils.create_vector_graph(v)])

## Match to plane in full size cloud

In [24]:
# now match to the original full size point cloud

cl.paint_uniform_color([0.5,0.5,0.5])
for norm, colour in zip(plane_normals, colours):
    dot_match = np.abs(1-np.asarray(cl.points).dot(norm))
    for i in range(len(dot_match)):
        if dot_match[i] <0.04:
            cl.colors[i] = colour

# add normals for clearer viewing
if not cl.has_normals():
    cl.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.04 * 2, max_nn=30))
o3d.visualization.draw_geometries([cl])